In [13]:
# Trying to fix the range doppler stuff in here

import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftshift # Might not need this
from radar_ffts import range_doppler_fft, range_doppler_sum
from cfar import cfar, clean_cfar
from radar_functions import get_measurement_parameters, cfar_map, save_range_doppler_map, save_cfar_map, get_data_files

c = 299792458 # metres per second - need this

In [14]:
session2_preface = '../4022S_data/session2/'

tennis_ball_tests = get_data_files(session2_preface) #just using this test
walking_away_test = h5py.File('../4022S_data/undergrad_testset/Experiment_moving_target_back_data.hdf5', 'r')

range_bin_size = 0.1953138069708251


frame_data = walking_away_test[f'Sensors/TI_Radar/Data/Frame_5/frame_data']
fftd_frame_data = range_doppler_fft(frame_data, 0, 0)



plotted_fftd_frame_data = range_doppler_sum(fftd_frame_data)
print(plotted_fftd_frame_data.dtype)

float64


In [15]:
def make_map(plotted_fftd_frame_data, range_bin_size):
    plt.figure()
    plt.imshow(plotted_fftd_frame_data, aspect='auto', cmap='jet')
    plt.title('Range-Doppler Map')
    plt.xlabel('Doppler')
    plt.ylabel('Range')
    plt.colorbar(label='Power (dB)')
    # Get current y-ticks and labels
    y_ticks = plt.gca().get_yticks()
    plt.gca().set_yticklabels(y_ticks[::1]*range_bin_size) #TODO: Figure out how to relabel the data, not just the ticks
    plt.show()

def range_doppler_map(hdf5_file_path, frame, range_bin_size, make_map): # This started as a clone from radar_functions
    
    """Generates a range doppler map of hdf5 radar data. Can generate a plot (make_map = 1) or just the data (make_map = 0)
        -> make_map = 1 plots the heatmap, heat_map = 0 skips it
        -> save_map = 1 saves the map as a png, save_map = 0 skips it"""
    
    frame_data = hdf5_file_path[f'Sensors/TI_Radar/Data/Frame_{frame}/frame_data']
    range_pad = 0
    doppler_pad = 0

    fftd_frame_data = range_doppler_fft(frame_data, range_pad, doppler_pad)
    plotted_fftd_frame_data = range_doppler_sum(fftd_frame_data)
    plotted_fftd_frame_data=np.flip(plotted_fftd_frame_data, 0)

    if make_map:
        make_map(plotted_fftd_frame_data, range_bin_size)
        
    return plotted_fftd_frame_data

In [17]:
range_doppler_map(tennis_ball_tests[0], 6, 1, True)

TypeError: 'bool' object is not callable